# PTM with 6 strain rates

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("mime")
import h5py
from scipy.optimize import curve_fit
import lmfit

colors = ['#bb0000', '#00bb00', "#0000bb", '#bbbb00', '#bb00bb', "#00bbbb", '#bbbbbb', '#770000', '#007700', "#000077", '#777700', '#770077', "#007777", '#777777', '#440000', '#004400', "#000044", '#444400', '#440044', "#0044444", '#444444','#000000']

In [ ]:
baseSize = (8, 6)  # Base size of a subplot

def sbPlot(n):
    if (n == 1): return 1, 1
    if (n <= 2): return 1, 2
    if (n <= 4): return 2, 2
    if (n <= 6): return 3, 2
    if (n <= 9): return 3, 3
    if (n <= 12): return 4, 3
    return 0, 0

def sbPlotSize(n):
    x, y = sbPlot(n)
    return baseSize[0] * y, baseSize[1] * x

In [ ]:
h5f = h5py.File('../GleebleData.h5','r')
allData = h5f['all'][:]
shortData = h5f['short'][:]
h5f.close()

In [ ]:
allData.shape, shortData.shape

Remove first point of each curve, where $\varepsilon^p=0$

In [ ]:
allData = allData[allData[:,0] != 0]
shortData = shortData[shortData[:,0]!=0]
identData = shortData

In [ ]:
strains = np.unique(identData[:,0])
allStrains = np.unique(allData[:,0])
epsps = np.unique(identData[:,1])
temperatures = np.unique(identData[:,2])
nEps = len(strains)
nEpsp = len(epsps)
nTemp = len(temperatures)
#strains, epsps, temperatures, nEps, nEpsp, nTemp

# Identification of the PTM parameters
$$\sigma^y(\varepsilon^p,\dot\varepsilon,T) = \left(\sum_{i=0}^{q}{A_i\varepsilon^{p^i}}\right) \exp\left[\left(\sum_{j=0}^{r}{B_j\varepsilon^{p^j}}\right)\left(T-T_0\right) + \left(\sum_{k=0}^{s}\left(\sum_{l=0}^{t}{C_k^l\varepsilon^{p^l}} \right)\left(T-T_0\right)^k \right)\ln\left( \frac{\dot\varepsilon}{\dot{\varepsilon}_0}\right)\right]$$

In [ ]:
T0 = temperatures[0]
epsp0 = epsps[0]
Tm = 1460
q=5
r=5
s=1
t=5

In [ ]:
def genParams(params, label, order, srange=None):
    for i in range(order+1):
        if srange == None :
            params.add(label+str(i), value=0)
        else:
            params.add(label+str(i), value=0, min=-srange, max= +srange)

In [ ]:
def polyFunc(eps, opt):
    res = 0
    i = 0
    for k in opt.keys():
        res += opt[k]*eps**i
        i += 1
    return res

In [ ]:
refSRdata = identData[identData[:,1]==epsp0]
comSig = np.concatenate((refSRdata, np.log(refSRdata[:,3].reshape(refSRdata[:,3].shape[0], 1))), axis=1)

In [ ]:
def constitutiveLaw(T, I1, S1):
    return I1 + S1*T 

In [ ]:
I1 = []
S1 = []
for i in list(strains):
    sbdata = comSig[comSig[:,0]==i]
    popt, pcov = curve_fit(constitutiveLaw, sbdata[:,2]-T0, sbdata[:,4])
    I_i, S_i = popt
    I1.append(I_i)
    S1.append(S_i)
J = np.exp(I1)

In [ ]:
AI = lmfit.Parameters()
genParams(AI, 'A', q)

def objA(opt):
    return J - polyFunc(strains, opt)

fitA = lmfit.minimize(objA, AI)
fitA.params

In [ ]:
BI = lmfit.Parameters()
genParams(BI, 'B', r)

def objB(opt):
    return S1 - polyFunc(strains, opt) 

fitB = lmfit.minimize(objB, BI)
fitB.params

In [ ]:
def indenfunctionS2(t, S2):
    return S2*t 

In [ ]:
C0 = []
C1 = []
for i in list(strains):
    sbdata = identData[identData[:,0]==i]
    S2 = []
    for j in list(temperatures):
        sbdata1 = sbdata[sbdata[:,2]==j]
        sig_j = np.log(sbdata1[:,3]) - np.log(polyFunc(i, fitA.params)) - polyFunc(i, fitB.params) * (j - T0)
        popt, pcov = curve_fit(indenfunctionS2, np.log(sbdata1[:,1]/epsp0), sig_j)
        S_j = popt
        S2.append(S_j)
    S2_params = np.polyfit(temperatures-T0, S2, 1)
    y25, y24 = S2_params
    C0.append(y24[0])
    C1.append(y25[0])

In [ ]:
C0I = lmfit.Parameters()
genParams(C0I, 'C0', t)

def objC0(opt):
    return C0 - polyFunc(strains, opt)

fitC0 = lmfit.minimize(objC0, C0I)
fitC0.params

In [ ]:
C1I = lmfit.Parameters()
genParams(C1I, 'C1', t)

def objC1(opt):
    return C1 - polyFunc(strains, opt)

fitC1 = lmfit.minimize(objC1, C1I)
fitC1.params

In [ ]:
def PTMLaw(eps, epsp, T):
    return polyFunc(eps, fitA.params) * np.exp(polyFunc(eps, fitB.params)*(T - T0) + (polyFunc(eps, fitC0.params) + polyFunc(eps, fitC1.params)*(T-T0)) * np.log(epsp/epsp0))

In [ ]:
# Plot the curves
from matplotlib.lines import Line2D
def create_dummy_line(**kwds):
    return Line2D([], [], **kwds)

plt.figure(figsize = sbPlotSize(nEpsp))
plt.rc('text', usetex = True)
idx = 1
plt.subplots_adjust(hspace = 0.3)
for epsp in list(epsps):
    xs, ys = sbPlot(nEpsp)
    plt.subplot(xs, ys, idx)
    sbdata = shortData[shortData[:,1]==epsp]
    cl =0
    for temp in list(temperatures):
        sbdata1 = sbdata[sbdata[:,2]==temp]
        plt.plot(sbdata1[:,0], sbdata1[:,3], colors[cl], marker = 's', markersize = 5, linestyle = 'none')
        plt.plot(strains, PTMLaw(strains, epsp, temp), colors[cl], linewidth = 2.5)
        plt.rcParams['xtick.labelsize'] = 16
        plt.rcParams['ytick.labelsize'] = 16
        cl +=1
    plt.xlim(0, 0.7)
    plt.ylim(bottom=0)
    plt.xlabel(r'strain $\varepsilon$', fontsize = 16) # Labels the x axis
    plt.ylabel(r'flow stress $\sigma^y$ (MPa)', fontsize = 16) # Labels the y axis
    plt.title(r'strain rate $\dot{\varepsilon} = ' + str(epsp) + '$ s$^{-1}$', fontsize = 16) # Self explicit command
    idx += 1
    
legendLines = []
cl = 0
for temp in list(temperatures):
    legendLines.append((r'$T=$' + str(int(temp)) + r'$^{\circ}$C', {'color':colors[cl], 'linestyle':'-', 'linewidth':2.5, 'marker':'s'}))
    cl += 1

plt.legend([create_dummy_line(**l[1]) for l in legendLines],[l[0] for l in legendLines], 
           loc = 'upper center', fontsize = 12, ncols = 5, bbox_to_anchor = (0.0, -0.2), shadow = False)

plt.savefig("CompExpPTM.svg")
plt.show()

In [ ]:
ARstress = PTMLaw(allData[:,0], allData[:,1], allData[:,2])

In [ ]:
data = np.concatenate((allData[:,0:3],ARstress.reshape((ARstress.shape[0],1))),axis=1)
h5f = h5py.File('PTM-6.h5','w')
h5f.create_dataset('data', data = data)
h5f.close()

In [ ]:
EAAR = np.sum(np.abs((allData[:,3] - ARstress)/(allData[:,3])))*100/ARstress.shape[0]
print("EAAR = %.2f" %(EAAR) + ' %')

In [ ]:
RMSE = np.sqrt(np.sum((allData[:,3] - ARstress)**2)/ARstress.shape[0])
print('RMSE = %.2f' %(RMSE)+' MPa')

In [ ]:
def outOf(val, i):
    if ((val<-i) or (val>i)): return True
    return False

def conv (v, d):
    va = abs(v)
    e = int(np.log10(va))
    if (va < 1): e-=1
    if outOf(e,2):
        a = v * 10**(-e)
        v = np.round(a * 10**d)/10**d
        return str(v)+'\\times 10^{'+str(e)+'}'
    v = np.round(v * 10**d)/10**d
    return str(v)

l1=list(fitA.params.values())
l2=list(fitB.params.values())
l3=list(fitC0.params.values())
l4=list(fitC1.params.values())

for i in range(np.max([len(l1),len(l2),len(l3),len(l4)])):
    s = ''
    if i < (len(l1)):
        s += l1[i].name+'='+ str(conv(l1[i].value,4))
    s+=' & '
    if i < (len(l2)):
        s += l2[i].name+'='+ str(conv(l2[i].value,4))
    s+=' & '
    if i < (len(l3)):
        s += l3[i].name+'='+ str(conv(l3[i].value,4))
    s+=' & '
    if i < (len(l4)):
        s += l4[i].name+'='+ str(conv(l4[i].value,4))
    s+='\\\\'    
    print(s)